Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fcharges-193353%2Fcharges.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/charges-193353)]*

## [``dyce``](https://posita.github.io/dyce/) translation of [Hypergardens’ solution to “Modelling ‘rolling d6 >= charges left expends a charge’ mechanics with anydice”](https://rpg.stackexchange.com/a/195107/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["ipywidgets==8.1.3", "matplotlib==3.5.2", "anydyce==0.4.6"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H
from dyce.evaluation import explode
from fractions import Fraction

try:
    from functools import cache
except ImportError:
    from functools import lru_cache
    cache = lru_cache(maxsize=None)

def rolls_before_depleting_one_charge(charges: int, d: H) -> H:
    return explode(
        d.lt(charges),
        limit=Fraction(1, 250),
    )

@cache  # <-- probably not helpful until charges is pretty large (see below), but doesn't hurt
def expected_uses(charges: int, d: H = H(6)) -> H:
    return (
        H({0: 1}) if charges <= 0
        else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses(charges - 1)
    )

In [3]:
from anydyce import jupyter_visualize

d = 6
hs_by_n_left = {n: expected_uses(n, H(d)) for n in range(0, d + 1)}

jupyter_visualize(
    (
        (f"Expected uses on\nd{d} with {n} charges\nMean: {h.mean():.2f}; Std Dev: {h.stdev():.2f}", h)
        for n, h in hs_by_n_left.items()
    ),
    initial_burst_cmap_inner="Blues_r",
    initial_burst_zero_fill_normalize=True,
    initial_enable_cutoff=True,
)

In [4]:
%%timeit

def expected_uses_not_cached(charges: int, d: H) -> H:
    return H({0: 1}) if charges <= 0 else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses_not_cached(charges - 1, d)

[expected_uses_not_cached(n, H(6)) for n in range(50, 0, -1)]

160 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit

@cache
def expected_uses_cached(charges: int, d: H) -> H:
    return H({0: 1}) if charges <= 0 else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses_cached(charges - 1, d)

[expected_uses_cached(n, H(6)) for n in range(50, 0, -1)]

4.83 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
